In [ ]:
%pip install pycocotools numpy==1.16.0 opencv-python tqdm tensorboard tensorboardX pyyaml webcolors matplotlib

In [ ]:
import os
import sys
if "projects" not in os.getcwd():
    os.chdir('Yet-Another-EfficientDet-Pytorch')
    sys.path.append('.')
else:
    !git pull

In [ ]:
!cat projects/taco.yml

In [ ]:
!python train.py -c 0 -p taco --head_only True --lr 1e-3 --batch_size 16 --load_weights weights/efficientdet-d0.pth --num_epochs 50 --save_interval 500

In [ ]:
!python train.py -c 0 -p taco --head_only False --lr 1e-4 --batch_size 8 --load_weights last --num_epochs 100 --save_interval 500

In [ ]:
%cd logs/taco
weight_file = !ls -Art | grep efficientdet
%cd ../..
print(f"Latest weight file: {weight_file[-1]}")

In [ ]:
from coco_eval import evaluate_coco

# Adjust these paths according to your setup
img_path = 'datasets/taco/val/'
set_name = 'val'
weights_path = f'logs/taco/{weight_file[-1]}'

# Load your TACO dataset and model
# This part depends on how you've structured your dataset and model loading
# You may need to adapt it based on the COCO dataset structure

# Run evaluation
evaluate_coco(img_path, set_name, weights_path)

In [ ]:
import torch
from backbone import EfficientDetBackbone
import cv2
import matplotlib.pyplot as plt
import numpy as np
from efficientdet.utils import BBoxTransform, ClipBoxes
from utils.utils import preprocess, invert_affine, postprocess

# Load your TACO object list
obj_list = ['Plastic bag & wrapper', 'Bottle', 'Bottle cap', 'Can', 'Other plastic', 'Other metal', 'Carton', 'Cup', 'Lid', 'Paper', 'Plastic container', 'Plastic utensils', 'Pop tab', 'Straw', 'Styrofoam piece', 'Unlabeled litter']

# Load model and weights
compound_coef = 0
model = EfficientDetBackbone(compound_coef=compound_coef, num_classes=len(obj_list))
model.load_state_dict(torch.load(f'logs/taco/{weight_file[-1]}'))
model.eval()

# Load and preprocess an image
img_path = 'datasets/taco/val/your_test_image.jpg'  # replace with an actual image from your dataset
ori_imgs, framed_imgs, framed_metas = preprocess(img_path, max_size=512)

# Perform inference
with torch.no_grad():
    x = torch.from_numpy(framed_imgs[0]).unsqueeze(0).permute(0, 3, 1, 2)
    features, regression, classification, anchors = model(x)

    regressBoxes = BBoxTransform()
    clipBoxes = ClipBoxes()

    out = postprocess(x, anchors, regression, classification, regressBoxes, clipBoxes, 0.2, 0.2)
    out = invert_affine(framed_metas, out)

# Visualize results
for i in range(len(ori_imgs)):
    if len(out[i]['rois']) == 0:
        continue
    for j in range(len(out[i]['rois'])):
        (x1, y1, x2, y2) = out[i]['rois'][j].astype(np.int)
        cv2.rectangle(ori_imgs[i], (x1, y1), (x2, y2), (255, 255, 0), 2)
        obj = obj_list[out[i]['class_ids'][j]]
        score = float(out[i]['scores'][j])
        cv2.putText(ori_imgs[i], f'{obj}, {score:.3f}', (x1, y1 + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)

plt.figure(figsize=(15, 15))
plt.imshow(ori_imgs[0])
plt.axis('off')
plt.show()